In [2]:
import numpy as np
import pandas as pd
from pathlib import Path

In [3]:
train_df = pd.read_csv(Path('Resources/2019loans.csv'))
test_df = pd.read_csv(Path('Resources/2020Q1loans.csv'))

In [6]:
# Clean data, examine dfs

# train data
# c = train_df.columns.values.tolist()
# clean = []
# for col in c:
#  if train_df[col].dtype == "object":
#    clean.append(col)
# print(clean)
# train_df.describe
# test_df.shape
# train_X = train_df.drop([train_df[1], train_df[2])
test_df.iloc[1]
# train_df['loan_status']

#drop index columns, and results columns
train_df1 = train_df.drop(['Unnamed: 0', 'index'], axis=1)
test_df1 =test_df.drop(['Unnamed: 0', 'index'], axis=1)
print(train_df1.shape)
print(test_df1.shape)

(12180, 84)
(4702, 84)


In [8]:
# Convert categorical data to numeric and separate target feature for test data
test_df_dummies = pd.get_dummies(test_df1)
test_df_X = test_df_dummies.drop(['loan_status_low_risk', 'loan_status_high_risk'], axis=1)
print(test_df_X.shape)
test_df_y = test_df_dummies['loan_status_low_risk']


(4702, 91)


In [17]:
# Convert categorical data to numeric and separate target feature for train data
train_df_dummies = pd.get_dummies(train_df1)
# train_df_dummies.columns.values.tolist()
# train_df_dummies['loan_status_low_risk']
train_df_X = train_df_dummies.drop(['loan_status_low_risk', 'loan_status_high_risk'], axis=1)
print(train_df_X.shape)
train_df_y = train_df_dummies['loan_status_low_risk']
# train_df_y.shape

(12180, 92)


(12180,)

In [19]:
# add missing dummy variables to testing set
#what columns is missing?
# cat = train_df_X.columns.values.tolist()
# cleanc1 = []
# for col in cat:
#  if train_df_X[col].dtype == "uint8":
#    cleanc1.append(col)
# print(cleanc1)

# cat = train_df_X.columns.values.tolist()
# cleanc = []
# for col in cat:
#  if test_df_X[col].dtype == "uint8":
# #    cleanc.append(col)
#    print(col,test_df_X[col].dtype )
    
#missing columns 'debt_settlement_flag_Y'
# pd.get_dummies(df).reindex(columns = cats, fill_value=0)

test_df_X1 = pd.get_dummies(test_df_X).reindex(columns = ['debt_settlement_flag_Y'], fill_value=0)
test_df_X2 = pd.concat([test_df_X,test_df_X1], axis=1, join="inner")
test_df_X2.shape


(4702, 92)

In [22]:
# model prep
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

y_label = LabelEncoder().fit_transform(train_df_y)
y_label
X_train, X_test, y_train, y_test = train_test_split(train_df_X, y_label, random_state=1)

In [39]:
# Train the Logistic Regression model on the unscaled data and print the model score
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

classifier.fit(X_train, y_train)

print(f"Training Data Score: {classifier.score(X_train, y_train)}")

# classifier.fit(X_test, y_test)

print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

# classifier.fit(test_df_X2,test_df_y)

print(f"Test Data Score: {classifier.score(test_df_X2, test_df_y)}")


Training Data Score: 0.6522167487684729
Testing Data Score: 0.6472906403940887
Test Data Score: 0.5161633347511697


C:\Users\Julio\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [38]:
# Train a Random Forest Classifier model and print the model score
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=1, n_estimators=92).fit(X_train, y_train)
print(f'Training Score: {clf.score(X_train, y_train)}')
print(f'Testing Score: {clf.score(X_test, y_test)}')

# clf = RandomForestClassifier(random_state=1, n_estimators=92).fit(test_df_X2, test_df_y)
print(f'Test Score: {clf.score(test_df_X2, test_df_y)}')


Training Score: 1.0
Testing Score: 0.7697865353037767
Test Score: 0.6278179498085921


In [32]:
# Scale the data
#train data
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_testing_scaled = scaler.transform(X_test)

#test data
X_test_scaled = scaler.transform(test_df_X2)

In [40]:
# Train the Logistic Regression model on the scaled data and print the model score
classifier = LogisticRegression()
classifier

classifier.fit(X_train_scaled, y_train)

print(f"Training Data Scaled Score: {classifier.score(X_train_scaled, y_train)}")

# classifier.fit(X_testing_scaled, y_test)

print(f"Testing Data Scaled Score: {classifier.score(X_testing_scaled, y_test)}")

# classifier.fit(X_test_scaled,test_df_y)

print(f"Test Data Scaled Score: {classifier.score(X_test_scaled, test_df_y)}")



Training Data Scaled Score: 0.7041050903119869
Testing Data Scaled Score: 0.6991789819376026
Test Data Scaled Score: 0.7579753296469588


C:\Users\Julio\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [37]:
# Train a Random Forest Classifier model on the scaled data and print the model score
clf = RandomForestClassifier(random_state=1, n_estimators=92).fit(X_train_scaled, y_train)
print(f'Training Scaled Score: {clf.score(X_train_scaled, y_train)}')
print(f'Testing Scaled Score: {clf.score(X_testing_scaled, y_test)}')

# clf = RandomForestClassifier(random_state=1, n_estimators=92).fit(X_test_scaled, test_df_y)
print(f'Test ScaledScore: {clf.score(X_test_scaled, test_df_y)}')

Training Scaled Score: 1.0
Testing Scaled Score: 0.7694581280788177
Test ScaledScore: 0.6290940025521055
